In [4]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from auto_ts import auto_timeseries
from pmdarima.arima import auto_arima,ADFTest
from autots import AutoTS

from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings


## On va essayer de construire un modèle pour chaque famille produit dans ce notebook 

In [2]:
def r2_ajuste(r2,n,p):

    return  1 - (  ((1 - r2) * (n - 1)) / (n - p - 1)  )

def determine_petit_truc(series):
    min_positive_value = series[series > 0].min()
    val = 0.00001
    return min_positive_value * val

# Fonction pour tester la stationnarité avec le test ADF
def test_stationarity(series):
    result = adfuller(series.dropna())
    return result[1]  # Retourne la p-value

# Fonction pour appliquer la transformation logarithmique
def apply_log(series, p):
    return np.log(series + p)

## reconstruction pour un dataframe 
def deDiff(orig, diff,order,log_order):
    dfout = pd.DataFrame(index=orig.index, columns=orig.columns)
    cols = orig.columns

    for family in cols:
        #affectation des premières valeurs 
        dfout[family].iloc[:order] = orig[family].iloc[:order]
        # recontruction du reste
        for i in range(order, len(orig)):
            dfout[family].iloc[i] = dfout[family].iloc[i-order]  +  diff[family].iloc[i-order]
        dfout[family]
        if log_order == 1:
            dfout = np.exp(dfout)
    return dfout
## reconstruction pour une colonne de dataframe 

def deDiffFamily(orig, diff, order,log_order):
    """
    Fonction poure reconstruire les colonnes de dataframes à différenciation
    orig = la colonne originale
    diff = la colonne des valuers différenciées
    order = l'ordre de différentiation de la colone
    log_order = 1 si on a utilisé un logarithme et 0 sinon
    """
    dfout = pd.Series(index=orig.index, dtype=orig.dtype)
    dfout.iloc[:order] = orig.iloc[:order]
    for i in range(order, len(orig)):
        dfout.iloc[i] = dfout.iloc[i-order] + diff.iloc[i-order]
    if log_order == 1:
        dfout = np.exp(dfout)
    return dfout

## Chargement des données 

In [9]:
# Charger les données des séries différenciées à partir d'un fichier CSV
df_differentiated = pd.read_csv('series_differenciees.csv')
df_transfo = pd.read_csv('transformations_appliquees.csv')
df_transfo.set_index('Famille', inplace=True)

## Auto arima pour trouver le meilleur modèle 

In [3]:
# Ignorer les avertissements liés à l'auto-ARIMA pour garder le code propre
warnings.filterwarnings("ignore")

best_models = {}
      #  transfo_family = dictTransformations.loc[family]
      #  diff_order = transfo_family['ordreP']  
# Parcourir chaque colonne (chaque série temporelle)

for col in df_differentiated.columns:
    print(f"Processing series: {col}")
    series = df_differentiated[col].dropna()  # Supprimer les éventuels NaN
    diff_fam = df_transfo.loc[col]
    # Appliquer l'auto-ARIMA pour trouver le meilleur modèle
    model = auto_arima(
        series,
        start_p=1,
        start_q=1,
        max_p=5, 
        max_q=5,
        d = int(diff_fam['Différenciations']) ,
        seasonal=False,  # Modèle non saisonnier
        error_action='warn',
        stepwise=True,  # Méthode plus rapide en recherchant de manière itérative
        trace=True  # Afficher le processus de sélection du modèle
    )
    best_models[col] = model

# Affichage des modèles trouvés
for col, model in best_models.items():
    print(f"\nBest model for {col}:")
    print(model.summary())


Processing series: A
Performing stepwise search to minimize aic


 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=1423.554, Time=0.08 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=1520.395, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=1461.448, Time=0.00 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=1480.065, Time=0.02 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=1518.438, Time=0.00 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=1387.848, Time=0.02 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=1411.114, Time=0.01 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=1387.935, Time=0.09 sec
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=1385.180, Time=0.10 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=1416.249, Time=0.05 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=1378.658, Time=0.12 sec
 ARIMA(4,2,2)(0,0,0)[0] intercept   : AIC=1336.529, Time=0.17 sec
 ARIMA(4,2,1)(0,0,0)[0] intercept   : AIC=1348.876, Time=0.12 sec
 ARIMA(5,2,2)(0,0,0)[0] intercept   : AIC=1338.515, Time=0.28 sec
 ARIMA(4,2,3)(0,0,0)[0] intercept   : AIC=1333.918, Time=0.15 sec
 ARIMA(3,2

## Calcul des métriques pour le baseline model

#### SMAPE : Symetric Mean Absolute Percentage Error
#### Adjusted SMAPE : Adjusted Symmetric Mean Absolute Percentage Error
#### MAPE : Mean Absolute Percentage Error
#### MAAPE : Mean Arctangent Absolute Percentage Error

In [8]:
# Fonction pour calculer SMAPE
def smape(y_true, y_pred):
    smape_val = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    return smape_val

# Fonction pour calculer l'Adjusted SMAPE (ASMAPE)
def adjusted_smape(y_true, y_pred):
    adjsmape = 1/y_true.size  * np.sum( np.abs(y_pred - y_true) / ( np.abs(y_true) + np.abs(y_pred) )*100 )
    oldadjsmape = 100 * np.mean(np.abs(y_pred - y_true) / (0.5 * (np.abs(y_true) + np.abs(y_pred))))
    return adjsmape
# Fonction pour calculer le MAAPE
def maape(actual, forecast):
    maape = np.mean(np.arctan(np.abs((actual - forecast) / actual))) * (180/np.pi)
    return maape

# Créer un dictionnaire pour stocker les métriques
metrics = {}

# Parcourir chaque modèle ajusté
for col, model in best_models.items():
    # Obtenir la série originale (sans NaN)
    series = df_differentiated[col].dropna()
    n = 3
    # Diviser les données
    train = series.iloc[:-n]
    test = series.iloc[-n:]

    # Réajuster le modèle sur l'ensemble d'entraînement
    model.fit(train)
    
    # Faire des prévisions sur l'ensemble de test
    predictions = model.predict(n_periods=len(test))
    
    # Calculer les métriques
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    maape_value = maape(test, predictions)
    smape_value = smape(test, predictions)
    asmape_value = adjusted_smape(test, predictions)
    
    # Enregistrer les métriques
    metrics[col] = {
        "MAAPE": maape_value,
        "MAPE": mape,
        "SMAPE": smape_value,
        "ASMAPE": asmape_value
    }

# Afficher les métriques pour chaque série
for col, metric in metrics.items():
    print(f"\nMetrics for {col}:")
    print(f"MAAPE: {metric['MAAPE']}°")
    print(f"MAPE: {metric['MAPE']}%")
    print(f"SMAPE: {metric['SMAPE']}%")
    print(f"ASMAPE: {metric['ASMAPE']}%")


Metrics for A:
MAAPE: 56.363685758427806°
MAPE: 163.1070670390797%
SMAPE: 87.2694121936747%
ASMAPE: 43.63470609683735%

Metrics for B:
MAAPE: 26.232453911064557°
MAPE: 57.886483607682905%
SMAPE: 44.547599187713246%
ASMAPE: 22.273799593856626%

Metrics for C:
MAAPE: 27.570885599567525°
MAPE: 52.37269554344234%
SMAPE: 48.43809698201678%
ASMAPE: 24.219048491008394%

Metrics for D:
MAAPE: 70.82965889278489°
MAPE: 1420.720772546767%
SMAPE: 140.29328114039316%
ASMAPE: 70.14664057019658%

Metrics for E:
MAAPE: 40.304589767189896°
MAPE: inf%
SMAPE: 94.35229898102273%
ASMAPE: 47.176149490511364%

Metrics for F:
MAAPE: 45.93497352530506°
MAPE: 111.1697920540745%
SMAPE: 157.59124196592848%
ASMAPE: 78.79562098296424%

Metrics for G:
MAAPE: 25.059221604454795°
MAPE: 49.65309884675395%
SMAPE: 39.168751368881935%
ASMAPE: 19.584375684440964%

Metrics for H:
MAAPE: 24.040978900125875°
MAPE: 49.19161649206387%
SMAPE: 83.12882311103554%
ASMAPE: 41.564411555517765%

Metrics for I:
MAAPE: 42.0037956105895